In [1]:
from datasets import load_dataset

ds = load_dataset("RZ412/PokerBench")

In [38]:
ds.shape

{'train': (563200, 2), 'test': (11000, 2)}

In [39]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENROUTER_API_KEY")
ds_key = os.getenv("DEEPSEEK_API_KEY")

In [40]:
test_data = ds['test']

In [41]:
import requests
def query_llm(prompt, model):
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "prompt": prompt,
       # "temperature": 0.1,
        #"top_p": 0.95
    }
    response = requests.post("https://openrouter.ai/api/v1/completions", headers=headers, json=payload)
    response_data = response.json()
    if "choices" not in response_data:
        print(response_data)
        return
    txt = response.json()["choices"][0]["text"]
    return txt

In [51]:
from openai import OpenAI

client = OpenAI(api_key=api_key, base_url="https://openrouter.ai/api/v1")

def query_deepseek(prompt):
    try:
        response = client.chat.completions.create(
            model="deepseek/deepseek-r1",
            messages=[{"role": "user", "content": prompt}],
            #temperature=0.1, 
            #top_p=0.95
        )
        print("deekseek response: ", response)
        if response.choices:
            return response.choices[0].message.content
        else:
            print("No response choices available.")
            return None

    except Exception as e:
        print(f"Error querying DeepSeek API: {e}")
        return None


In [43]:
model_deepseek = "deepseek/deepseek-r1"
model_llama = "meta-llama/llama-3.3-70b-instruct"
model_gemini = "google/gemini-2.0-flash-lite-preview-02-05:free"

In [33]:
instructions = test_data['instruction']
outputs = test_data['output']

In [53]:
ans = query_llm2(instructions[0])
print(ans)

Your optimal action is: bet 4 chips.


In [ ]:
import numpy as np

np.random.seed(42)

test_data_array = np.array(test_data)

random_indices = np.random.choice(len(test_data_array), size=1000, replace=False)

random_test_sample = test_data_array[random_indices]


In [101]:
instructions = test_data['instruction']
outputs = test_data['output']

predictions = [query_llm(instruction, model_gemini) for instruction in instructions[:18]]

aa, em = evaluate_model(predictions, outputs[:18])

print(f"Action Accuracy (AA): {aa:.2f}%")
print(f"Exact Match Accuracy (EM): {em:.2f}%")

Check
Check
Check
Bet 10 chips
Check
Call
Call
Check
Call
Call
Call
Bet 20 chips
Check
Check
Check
Fold
Call
Call
Action Accuracy (AA): 83.33%
Exact Match Accuracy (EM): 77.78%
